# WASD Abnormal Signal Generation (Example)

This notebook demonstrates clean reference implementations of three synthetic anomaly types:
- tone
- chirp
- pulse train

It also visualizes each signal in time and frequency domains.
        


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.signal import spectrogram

plt.rcParams["figure.figsize"] = (10, 4)
        


## 1) Global parameters


In [ ]:
FS = 1_300_000          # sample rate [Hz]
DURATION = 1.0          # signal length [s]
N = int(FS * DURATION)  # number of complex baseband samples

t = np.arange(N) / FS
        


## 2) Utilities


In [ ]:
def gate_window(num_samples: int, start: int, end: int) -> np.ndarray:
    g = np.zeros(num_samples, dtype=float)
    start = max(0, start)
    end = min(num_samples, end)
    if end > start:
        g[start:end] = 1.0
    return g


def plot_time_and_spectrogram(x: np.ndarray, fs: float, title: str, nperseg: int = 3200, noverlap: int = 2390):
    fig, axes = plt.subplots(1, 2, figsize=(14, 4))
    n_show = min(20_000, len(x))
    axes[0].plot(np.real(x[:n_show]), label="I")
    axes[0].plot(np.imag(x[:n_show]), label="Q", alpha=0.8)
    axes[0].set_title(f"{title} (time-domain preview)")
    axes[0].set_xlabel("Sample index")
    axes[0].set_ylabel("Amplitude")
    axes[0].grid(True, alpha=0.3)
    axes[0].legend()

    f, tt, sxx = spectrogram(
        x,
        fs=fs,
        nperseg=nperseg,
        noverlap=noverlap,
        return_onesided=False,
        mode="magnitude",
        scaling="spectrum",
    )
    sxx = np.fft.fftshift(sxx, axes=0)
    f = np.fft.fftshift(f)

    im = axes[1].imshow(
        20 * np.log10(np.maximum(sxx, 1e-12)),
        aspect="auto",
        origin="lower",
        extent=[tt[0], tt[-1], f[0] / 1e3, f[-1] / 1e3],
        cmap="viridis",
    )
    axes[1].set_title(f"{title} (spectrogram)")
    axes[1].set_xlabel("Time [s]")
    axes[1].set_ylabel("Frequency [kHz]")
    fig.colorbar(im, ax=axes[1], label="Magnitude [dB]")
    plt.tight_layout()
    plt.show()
        


## 3) Tone anomaly


In [ ]:
def tone_anomaly(fs=FS, duration=DURATION, f0=0.0, start=300_000, end=900_000):
    n = int(fs * duration)
    tt = np.arange(n) / fs
    g = gate_window(n, start, end)
    return g * np.exp(1j * 2 * np.pi * f0 * tt)


tone = tone_anomaly()
plot_time_and_spectrogram(tone, FS, "Tone anomaly")
        


## 4) Chirp anomaly


In [ ]:
def chirp_anomaly(fs=FS, duration=DURATION, f_start=-FS/3, f_end=FS/3, start=200_000, end=1_100_000):
    n = int(fs * duration)
    tt = np.arange(n) / fs
    k = (f_end - f_start) / duration
    phase = 2 * np.pi * (f_start * tt + 0.5 * k * tt**2)
    g = gate_window(n, start, end)
    return g * np.exp(1j * phase)


chirp_sig = chirp_anomaly()
plot_time_and_spectrogram(chirp_sig, FS, "Chirp anomaly")
        


## 5) Pulse-train anomaly


In [ ]:
def pulse_anomaly(
    fs=FS,
    duration=DURATION,
    f_shift=100_000.0,
    pulse_spacing=100_000,
    sigma=10.0,
    start=100_000,
    end=600_000,
):
    n = int(fs * duration)
    tt = np.arange(n) / fs
    x = np.arange(n, dtype=float)

    centers = np.arange(0, n, pulse_spacing, dtype=float)
    train = np.zeros(n, dtype=float)
    for c in centers:
        train += np.exp(-0.5 * ((x - c) / sigma) ** 2)

    g = gate_window(n, start, end)
    env = train * g
    return env * np.exp(1j * 2 * np.pi * f_shift * tt)


pulse = pulse_anomaly()
plot_time_and_spectrogram(pulse, FS, "Pulse anomaly")
        


## 6) Optional composition with measured IQ

If you have measured IQ `x_meas`, you can synthesize a composite sample:

```python
alpha = 0.2
x_mix = x_meas + alpha * chirp_sig
```

Then generate a spectrogram and labels according to your dataset pipeline.
        
